# Rede Neural de Multilayer Perceptron (MLP)
## Autor: Mariano F.M.A.S.
### Data: 06/07/2023


# Rede Neural Artificial 

Uma Rede Neural Artificial (ANN) é um modelo de aprendizado de máquina inspirado na estrutura e funcionamento do cérebro humano. Ela é composta por um grande número de unidades de processamento, chamadas neurônios ou nós, que estão interconectados por links chamados conexões. Cada conexão entre os neurônios carrega um peso, que pode ser pensado como a "força" da conexão. Esses pesos são ajustados durante o processo de treinamento para melhorar a precisão do modelo.
 

<img src="https://mriquestions.com/uploads/3/4/5/7/34572113/perceptron-with-neuron_1.png" alt="Comparação do Neurônio Artificial com o Neurônio Humano." width="600"  >
 


A **Rede Neural de Multilayer Perceptron (MLP)** é um tipo específico de ANN que é composta por três ou mais camadas de neurônios: uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída. Cada neurônio em uma camada está conectado a todos os neurônios na camada seguinte, daí o termo "totalmente conectado".

Aqui está uma descrição mais detalhada das camadas em uma MLP:

1. **Camada de entrada:** Esta é a camada que recebe os dados de entrada. O número de neurônios nesta camada geralmente corresponde ao número de características nos dados de entrada.

2. **Camadas ocultas:** Estas são as camadas entre a camada de entrada e a camada de saída. Cada neurônio em uma camada oculta recebe entradas de todos os neurônios na camada anterior, aplica uma função de ativação (como a função sigmoid, tanh ou ReLU), e passa o resultado para a próxima camada. O número de camadas ocultas e o número de neurônios em cada camada são parâmetros que podem ser ajustados para melhorar o desempenho do modelo.

3. **Camada de saída:** Esta é a camada que produz a saída do modelo. O número de neurônios nesta camada geralmente corresponde ao número de classes no problema de classificação (para tarefas de classificação) ou a 1 (para tarefas de regressão).

Durante o treinamento, a MLP usa um algoritmo chamado backpropagation para ajustar os pesos das conexões. O objetivo é minimizar a diferença entre a saída prevista pela rede e a saída real (ou seja, o erro) para todos os exemplos de treinamento.

<img src="https://hashdork.com/wp-content/uploads/2022/04/Neural-Network.png" alt="Rede Neural de Multilayer Perceptron." width="500"  >
 


In [ ]:
import numpy as np
import scipy


In [ ]:
import torch
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, matthews_corrcoef, cohen_kappa_score, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.metrics import average_precision_score

from itertools import cycle 
from sklearn.preprocessing import label_binarize
   
# Checking GPU is available 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Training on device: ", device)
# Set the random seed
random_seed = 11




# Carregando os dados

Ao criar um modelo de aprendizado de máquina, é comum dividir o conjunto de dados total em três subconjuntos: treinamento, validação e teste. Cada um desses conjuntos tem um propósito específico no processo de criação e avaliação do modelo.

1. **Conjunto de Treinamento**: Este é o subconjunto de dados usado para treinar o modelo. O algoritmo de aprendizado de máquina "aprende" ajustando seus parâmetros para minimizar o erro entre suas previsões e os valores reais para os exemplos no conjunto de treinamento.

2. **Conjunto de Validação**: Este é um subconjunto de dados que é usado para ajustar os hiperparâmetros do modelo, como a taxa de aprendizado ou a profundidade de uma árvore de decisão. O conjunto de validação é usado para evitar o sobreajuste durante o treinamento, permitindo que você ajuste o modelo com base em seu desempenho em dados não vistos durante o treinamento. Em outras palavras, você usa o conjunto de validação para "validar" as escolhas que fez durante o treinamento e ajustar o modelo de acordo.

3. **Conjunto de Teste**: Este é um subconjunto de dados que é mantido separado e não é usado durante o processo de treinamento. Depois que o modelo foi treinado e ajustado, o conjunto de teste é usado para avaliar o desempenho do modelo. Isso fornece uma estimativa imparcial de como o modelo provavelmente se sairá em dados não vistos no mundo real.

A divisão típica dos dados pode ser 70% para treinamento, 15% para validação e 15% para teste, mas isso pode variar dependendo do tamanho e da natureza do conjunto de dados. Além disso, técnicas como validação cruzada podem ser usadas para fazer uso mais eficiente dos dados disponíveis.

Embora tecnicamente seja possível usar o conjunto de validação como um conjunto de teste, isso não é recomendado e pode levar a uma avaliação imprecisa do desempenho do modelo.

O conjunto de validação é usado durante o processo de treinamento para ajustar os hiperparâmetros do modelo e evitar o sobreajuste. Isso significa que o modelo tem acesso indireto a esses dados durante o treinamento. Portanto, o desempenho do modelo no conjunto de validação pode ser otimista, pois o modelo foi ajustado para se sair bem nesses dados.

Por outro lado, o conjunto de teste é um conjunto de dados que o modelo nunca viu durante o treinamento ou a validação. Ele fornece uma avaliação imparcial do desempenho do modelo em dados não vistos. Isso é importante para ter uma ideia de como o modelo provavelmente se sairá em um cenário do mundo real.

Portanto, para uma avaliação adequada do desempenho do modelo, é melhor manter conjuntos de validação e teste separados. O conjunto de validação deve ser usado para ajustar o modelo durante o treinamento, e o conjunto de teste deve ser usado para avaliar o desempenho do modelo final.



In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
 

In [ ]:
# Importando os dados

# Dados de treinamento 
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
#Separa os entre train e test
train, val = train_test_split(data, test_size=0.2)

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

# Dados para validação do modelo
test_data  = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')



## Criando os dataloaders

No PyTorch, a classe `Dataset` é uma classe abstrata que representa um conjunto de dados. Ela é usada para carregar e pré-processar os dados. A classe `Dataset` deve ser estendida para fornecer métodos específicos para o conjunto de dados que você está usando.

A classe `Dataset` requer que você implemente pelo menos dois métodos:

1. `__len__`: Este método deve retornar o número total de amostras no conjunto de dados.

2. `__getitem__`: Este método deve receber um índice e retornar a amostra correspondente do conjunto de dados.

Aqui está um exemplo de como você pode definir uma subclasse de `Dataset` para um conjunto de dados de imagens:

 
Depois de definir a classe `Dataset`, você pode usá-la para criar um objeto `DataLoader`, que é um iterador que eficientemente carrega os dados em lotes e fornece muitas outras funcionalidades, como embaralhamento dos dados e carregamento de dados em paralelo.

In [ ]:
 
# Definindo a classe do dataset
class MNISTDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = torch.tensor(self.data.iloc[index, 1:].values/255, dtype=torch.float32) # /255 para normalizar os Pixels
        label = torch.tensor(self.data.iloc[index, 0], dtype=torch.long)
        return image, label

# Criando os dataloaders
train_dataset = MNISTDataset(train)
val_dataset = MNISTDataset(val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)



Se `shuffle=True`, os dados serão embaralhados antes de cada época. Isso é útil para garantir que o modelo não aprenda nada das sequências específicas dos dados de entrada. Em outras palavras, embaralhar os dados garante que o modelo não seja influenciado pela ordem dos exemplos de treinamento.

Se `shuffle=False`, os dados serão passados na mesma ordem a cada época. Isso pode ser útil em certos casos, como quando você está trabalhando com séries temporais e a ordem dos dados é importante.

Em geral, é uma boa prática embaralhar os dados de treinamento para garantir que o modelo seja robusto e não aprenda a fazer previsões com base na ordem dos exemplos.

In [ ]:
 
# Definindo a classe do dataset
class MNISTDataset_test(Dataset):
    def __init__(self, test_data):
        self.test_data = test_data
    
    def __len__(self):
        return len(self.test_data)
    
    def __getitem__(self, index):
        image = torch.tensor(self.test_data.iloc[index,:].values/255, dtype=torch.float32)
        return image

# Criando os dataloaders
test_dataset = MNISTDataset_test(test_data)
 
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


## Visualizando os Dados de Treinamento e Test 

In [ ]:
fig, axis = plt.subplots(3, 5, figsize=(10, 7))

for images, labels in train_loader:
    for i, ax in enumerate(axis.flat):
        image, label = images[i], labels[i]
        
        ax.imshow(image.view(28, 28), cmap='binary') # add image
        ax.set(title = f"{label}") # add label
    break  # exit the loop after the first batch


# Modelo MLP

`nn.Module` é uma das classes mais importantes no PyTorch e serve como base para a construção de todos os modelos de aprendizado de máquina. É uma classe base para todos os módulos de rede neural, e seus membros ou sub-classes podem ser usados como camadas de uma rede neural.

A classe `nn.Module` fornece muitas funcionalidades úteis. Aqui estão algumas das mais importantes:

1. **Gerenciamento de parâmetros**: `nn.Module` mantém o controle de todos os parâmetros (pesos e bias) de uma rede neural. Você pode usar o método `parameters()` para obter uma lista de todos os parâmetros do modelo que precisam de gradiente (ou seja, que serão atualizados durante o treinamento).

2. **Funcionalidades de GPU**: `nn.Module` facilita a transferência de todos os parâmetros do modelo para a GPU usando o método `to(device)`, onde `device` pode ser `cuda` para GPUs ou `cpu` para CPUs.

3. **Salvando e carregando o modelo**: `nn.Module` fornece métodos para salvar e carregar o modelo, que são `state_dict()` e `load_state_dict()`, respectivamente.

4. **Encapsulamento de camadas**: `nn.Module` permite que você defina suas próprias camadas e modelos personalizados, agrupando várias camadas juntas.

Para criar um modelo personalizado, você normalmente define uma subclasse de `nn.Module`, implementa o método `__init__` para definir as camadas do modelo, e implementa o método `forward` para especificar como o modelo processa as entradas.


O método `__init__` e o método `forward` são dois componentes fundamentais ao criar uma rede neural personalizada no PyTorch, estendendo a classe `nn.Module`.

**__init__**: Este é o construtor da classe. É aqui que você define as diferentes camadas da sua rede neural. Cada camada é representada por uma instância de uma classe `nn.Module` (como `nn.Linear`, `nn.Conv2d`, `nn.MaxPool2d`, etc.). Quando você cria uma instância de uma camada, você especifica os parâmetros dessa camada, como o número de entradas e saídas para uma camada linear, ou o tamanho do kernel e o número de canais para uma camada convolucional.


**forward**: Este método define como a rede neural realiza a propagação para frente (ou seja, como ela processa as entradas e calcula as saídas). Você implementa o método `forward` para usar as camadas que você definiu no `__init__` para transformar a entrada em uma saída.

## Construindo um Modelo MLP


In [ ]:
 
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 784)  # Primeira camada oculta com 784 neurônios
        self.fc2 = nn.Linear(784, 392)  # Segunda camada oculta com 392 neurônios
        self.fc3 = nn.Linear(392, 196)  # Terceira camada oculta com 196 neurônios
        self.fc4 = nn.Linear(196, 98)  # Quarta camada oculta com 98 neurônios
        self.fc5 = nn.Linear(98, 10)  # Camada de saída com 10 neurônios (para 10 classes) 

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Primeira camada oculta com função de ativação ReLU
        x = F.relu(self.fc2(x))  # Segunda camada oculta com função de ativação ReLU
        x = F.relu(self.fc3(x))  # Terceira camada oculta com função de ativação ReLU
        x = F.relu(self.fc4(x))  # Quarta camada oculta com função de ativação ReLU
        x = self.fc5(x)  # Camada de saída
        return x 
    
# Criando uma instância da rede neural
model = MNISTModel().to(device)
# Imprimindo a arquitetura da rede neural
print(model)



##  Função de Ativação

A é uma função matemática aplicada a um sinal de saída da soma ponderada dos neurônios. Ela é usada para determinar se o neurônio correspondente deve ser "ativado" ou não, com base nos valores de entrada recebidos.

O PyTorch fornece várias funções de ativação. Aqui estão algumas das mais comuns:

1. **ReLU (Rectified Linear Unit)**: É a função de ativação mais comumente usada em redes neurais e redes neurais convolucionais. Ela retorna o valor de entrada para entradas positivas e 0 para entradas negativas.

   Uso: `nn.ReLU()` ou `torch.relu(input)`

2. **Sigmoid**: Esta função de ativação mapeia os valores de entrada para o intervalo entre 0 e 1, tornando-a útil para a saída de probabilidades.

   Uso: `nn.Sigmoid()` ou `torch.sigmoid(input)`

3. **Tanh (Tangente Hiperbólica)**: Semelhante à função sigmoid, mas mapeia os valores de entrada para o intervalo entre -1 e 1.

   Uso: `nn.Tanh()` ou `torch.tanh(input)`

4. **Softmax**: Esta função de ativação é usada na camada de saída de redes neurais de classificação multiclasse. Ela transforma os valores de entrada em probabilidades que somam 1.

   Uso: `nn.Softmax(dim)` ou `torch.softmax(input, dim)`

5. **Leaky ReLU**: É uma variação da ReLU que permite pequenos valores negativos quando a entrada é menor que zero.

   Uso: `nn.LeakyReLU(negative_slope=0.01)` ou `F.leaky_relu(input, negative_slope=0.01)`

6. **ELU (Exponential Linear Unit)**: Semelhante à Leaky ReLU, mas a parte negativa é suavizada e aproxima-se de -1 para entradas negativas grandes.

   Uso: `nn.ELU(alpha=1.0)` ou `F.elu(input, alpha=1.0)`

7. **PReLU (Parametric ReLU)**: É uma versão da Leaky ReLU onde a inclinação para valores negativos é aprendida durante o treinamento.

   Uso: `nn.PReLU(num_parameters=1, init=0.25)`

8. **SELU (Scaled Exponential Linear Unit)**: É uma função de ativação que leva em conta a escala dos valores de entrada.

   Uso: `nn.SELU()` ou `F.selu(input)`

9. **GELU (Gaussian Error Linear Unit)**: É uma função de ativação que é usada em alguns modelos de redes neurais modernos, como o Transformer e o BERT.

   Uso: `nn.GELU()` ou `F.gelu(input)`

10. **Softplus**: É uma função de ativação que serve como uma alternativa suavizada à função ReLU.

    Uso: `nn.Softplus(beta=1, threshold=20)` ou `F.softplus(input, beta=1, threshold=20)`

Lembre-se de que a escolha da função de ativação depende do problema específico que você está tentando resolver e do tipo de rede neural que você está usando.

<img src="https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/62b18a8dc83132e1a479b65d_neural-network-activation-function-cheat-sheet.jpeg" alt="Funções de Ativação." width="800"  >


O PyTorch fornece uma variedade de camadas que você pode usar para construir sua rede neural. Aqui estão algumas das mais comuns:

1. **Linear**: Uma camada linear realiza uma transformação linear nos dados de entrada. É também conhecida como camada totalmente conectada.

2. **Convolutional (Conv1d, Conv2d, Conv3d)**: As camadas convolucionais são a pedra angular das redes neurais convolucionais (CNNs), que são comumente usadas para tarefas de processamento de imagem.

3. **Pooling (MaxPool1d, MaxPool2d, MaxPool3d, AvgPool1d, etc.)**: As camadas de pooling são usadas para reduzir a dimensionalidade dos dados de entrada.

4. **Dropout**: A camada de dropout é uma técnica de regularização que ajuda a prevenir o sobreajuste "desligando" aleatoriamente alguns neurônios durante o treinamento.

5. **BatchNorm (BatchNorm1d, BatchNorm2d, BatchNorm3d)**: As camadas de normalização em lote ajudam a acelerar o treinamento e melhorar a generalização, normalizando a saída de cada camada.

6. **Embedding**: A camada de incorporação é usada para converter entradas categóricas em vetores densos de tamanho fixo, geralmente para tarefas de processamento de linguagem natural.

7. **RNN, LSTM, GRU**: Estas são camadas recorrentes usadas para processar sequências de dados, como séries temporais ou texto.

8. **Transformer**: A camada Transformer é a base das redes de transformadores, que são usadas para tarefas de processamento de linguagem natural, como tradução automática.

9. **Softmax, LogSoftmax**: Estas são camadas de ativação usadas na camada de saída de uma rede neural para tarefas de classificação.

10. **ReLU, Sigmoid, Tanh, LeakyReLU, etc.**: Estas são camadas de ativação usadas para introduzir não-linearidades na rede.

Lembre-se de que a escolha da camada depende da tarefa específica que você está tentando resolver e do tipo de rede neural que você está usando. Além disso, muitas dessas camadas têm variantes (por exemplo, diferentes tipos de camadas de pooling ou normalização) que podem ser mais adequadas para certos tipos de dados ou tarefas.

# Função de Treinando

Definimos uma função para fazer o treinamento de um modelo qualquer.

Essa função vai treinar o modelo por infinitas epochs, até que a acurácia pare de aumentar após um determinado número de epocas dada por `patience`.

Após cada epoch, avaliamos o modelo usando o conjunto de validação.


In [ ]:
 
def train_model(model, criterion, optimizer, train_loader, val_loader, patience):
    best_accuracy = 0.0  # Inicialize a melhor acurácia como 0
    epochs_without_improvement = 0  # Contador para épocas sem melhoria
    #patience => Número de épocas para esperar, depois de obter a melhor acurácia, antes de parar
    model.train()
    epoch= 0
    
    while True:                      
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)   
             
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()  
            
        # Validação
        accuracy, validation_loss = validate(model, val_loader, criterion)
        print('Epoch [{}], Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, loss.item(), validation_loss, accuracy))
        epoch=epoch+1
        
        # Verifica se a acurácia melhorou
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            epochs_without_improvement = 0  # Reseta o contador
            torch.save(model.state_dict(), 'best_model_MLP.pth') # Salva o modelo com a melhor acurácia
        else:
            epochs_without_improvement += 1 # Incrementar o contador
         # Se a acurácia não melhorar por um número 'patience' de épocas, para de treinar
        if epochs_without_improvement == patience:
            print('Stopping training!')
            break
 

In [ ]:
def validate(model, val_loader, criterion):
    model.eval()  # Defina o modelo para o modo de avaliação
    correct = 0
    total = 0
    validation_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)   #Encontra o índice da probabilidade máxima
            # A função torch.max() retorna dois valores: o primeiro é o valor máximo encontrado em cada coluna do tensor
            #e o segundo é o índice do valor máximo em cada coluna. ex.: [0.2, 0.4, 0.9, 0.5] => (0.9, 2)= (valor máximo,índice)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = (correct / total) * 100
    validation_loss = validation_loss / len(val_loader)  # Media de loss

    model.train()  # Defin3 o modelo de volta ao modo de treinamento

    return accuracy, validation_loss



# Treinamento o Modelo MLP

In [ ]:
criterion = nn.CrossEntropyLoss()  # Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimizer
patience = 3  # Número de épocas para esperar antes de parar


train_model(model, criterion, optimizer, train_loader, val_loader, patience)


In [ ]:
# Inicialize o modelo
model = MNISTModel()

# Carregue o estado do modelo salvo
model.load_state_dict(torch.load('best_model_MLP.pth'))

model.eval()


# Avaliação do Modelo

##  Métricas de avaliação de modelos de classificação

Para calcular as metricas de avaliação usamos os sequintes valores:

**Verdadeiros Positivos (TP)**: Verdadeiros positivos são os casos em que o modelo previu a classe positiva corretamente. Em outras palavras, a classe real do exemplo era positiva e o modelo também previu a classe como positiva.

**Falsos Positivos (FP)**: Falsos positivos são os casos em que o modelo previu incorretamente a classe positiva. Isso significa que a classe real do exemplo era negativa, mas o modelo previu a classe como positiva. Isso é também conhecido como um erro do Tipo I.

**Verdadeiros Negativos (TN)**: Verdadeiros negativos são os casos em que o modelo previu a classe negativa corretamente. Em outras palavras, a classe real do exemplo era negativa e o modelo também previu a classe como negativa.

**Falsos Negativos (FN)**: Falsos negativos são os casos em que o modelo previu incorretamente a classe negativa. Isso significa que a classe real do exemplo era positiva, mas o modelo previu a classe como negativa. Isso é também conhecido como um erro do Tipo II.


Esses quatro valores formam a base da matriz de confusão. Além disso, várias métricas de avaliação, como precisão, recall e F1 score, são calculadas com base nesses valores. 

## Métricas


1. **Accuracy (Acurácia)**: É a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. É uma métrica útil quando as classes estão bem balanceadas, mas pode ser enganosa quando as classes estão desbalanceadas. A fórmula para a acurácia é:

   $$ \text{Accuracy} = \frac{\text{Verdadeiros Positivos (TP) + Verdadeiros Negativos (TN)}}{\text{Verdadeiros Positivos (TP) + Falsos Positivos (FP) + Verdadeiros Negativos (TN) + Falsos Negativos (FN)}} $$
   


2. **Precision (Precisão)**: É a proporção de previsões positivas que são realmente corretas. É uma métrica importante quando o custo de Falsos Positivos é alto. A fórmula para a precisão é:

   $$ \text{Precision} = \frac{\text{Verdadeiros Positivos (TP)}}{\text{Verdadeiros Positivos (TP) + Falsos Positivos (FP)}} $$


3. **Recall (Revocação ou Sensibilidade)**: É a proporção de positivos reais que foram identificados corretamente. É uma métrica importante quando o custo de Falsos Negativos é alto. A fórmula para o recall é:

   $$ \text{Recall} = \frac{\text{Verdadeiros Positivos (TP)}}{\text{Verdadeiros Positivos (TP) + Falsos Negativos (FN)}} $$


4. **F1 Score**: É a média harmônica entre a precisão e o recall. É uma métrica útil quando você precisa de um equilíbrio entre a precisão e o recall e há uma distribuição desigual de classes. A fórmula para o F1 Score é:

   $$ \text{F1 Score} = 2 * \frac{\text{Precision * Recall}}{\text{Precision + Recall}} $$


Essas métricas são comumente usadas para avaliar modelos de classificação e cada uma tem seus próprios pontos fortes e fracos, dependendo da situação específica.


5. **Coeficiente de Kappa de Cohen**

O coeficiente de Kappa de Cohen é uma estatística que é usada para medir a precisão de classificação em tarefas de classificação. É mais útil quando os dados são classificados por humanos, pois leva em consideração a possibilidade de o acordo ocorrer por acaso.

O coeficiente de Kappa de Cohen varia de -1 a 1. Um valor de 1 indica que há um acordo perfeito entre os classificadores. Um valor de 0 indica que o acordo é o mesmo que seria esperado por acaso. Um valor negativo indica que o acordo é pior do que o aleatório.

Aqui está como o coeficiente de Kappa de Cohen é geralmente interpretado:

- Valores ≤ 0: Nenhum acordo
- 0.01–0.20: Nenhum a um leve acordo
- 0.21–0.40: Acordo justo
- 0.41–0.60: Acordo moderado
- 0.61–0.80: Acordo substancial
- 0.81–0.99: Acordo quase perfeito
- 1: Acordo perfeito

O coeficiente de Kappa de Cohen é uma medida mais robusta do que a simples porcentagem de concordância, porque leva em consideração a possibilidade de o acordo ocorrer por acaso. Isso é especialmente importante quando os dados estão desequilibrados.


6. **Coeficiente de Correlação de Matthews (MCC)**

O Coeficiente de Correlação de Matthews (MCC) é uma medida de qualidade para problemas de classificação binária. Ele leva em consideração verdadeiros e falsos positivos e negativos e é geralmente considerado uma medida equilibrada, o que significa que pode ser usado mesmo se as classes estiverem de tamanhos muito diferentes.

O MCC é, em essência, uma correlação entre as observações reais e as previsões: um coeficiente de +1 representa uma previsão perfeita, 0 uma previsão média aleatória e -1 uma previsão inversa.

A fórmula para o MCC é:

$$ MCC = \frac{(TP*TN - FP*FN) }{ \sqrt{(TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)} }$$



O denominador da fórmula garante que o MCC sempre caia entre -1 e +1.

7. **Matriz de Confusão**

A Matriz de Confusão é uma tabela usada para descrever o desempenho de um modelo de classificação em um conjunto de dados para os quais os valores verdadeiros são conhecidos. Ela é chamada de matriz de confusão porque permite visualizar facilmente o tipo de confusão que o classificador está causando, mostrando onde o classificador está confundindo uma classe por outra.

 

Para uma classificação binária a matriz de confusão é uma tabela 2x2. As linhas da matriz representam as classes reais, enquanto as colunas representam as classes previstas pelo modelo. A matriz é organizada da seguinte forma:


|                     | **Previsto: Positivo** | **Previsto: Negativo** |
|---------------------|------------------------|------------------------|
| **Real: Positivo**  | Verdadeiros Positivos  | Falsos Negativos       |
| **Real: Negativo**  | Falsos Positivos       | Verdadeiros Negativos  |


A matriz de confusão é uma ferramenta poderosa para entender como seu modelo está performando e onde ele está cometendo erros. Além disso, muitas métricas de avaliação do modelo, como precisão, recall e F1 score, são calculadas com base nos valores da matriz de confusão.

7. **ROC Curve**

A curva ROC (Receiver Operating Characteristic) é uma ferramenta gráfica usada para avaliar o desempenho de um modelo de **classificação binária** ou diagnóstico. Ela foi desenvolvida durante a Segunda Guerra Mundial para a análise de sinais de radar e desde então tem sido usada em muitos outros campos.

A curva ROC é um gráfico de duas dimensões onde a taxa de verdadeiros positivos (TPR, True Positive Rate) é plotada no eixo Y e a taxa de falsos positivos (FPR, False Positive Rate) é plotada no eixo X.  

Cada ponto na curva ROC representa um par de valores (FPR, TPR) correspondentes a um limiar de decisão específico. O limiar de decisão é o valor a partir do qual decidimos se uma previsão é classificada como positiva ou negativa. Ao variar o limiar de decisão, obtemos diferentes pares de valores (FPR, TPR) que formam a curva ROC.

A linha diagonal do canto inferior esquerdo ao canto superior direito representa a curva ROC de um classificador aleatório (um classificador que faz previsões aleatórias). Um bom classificador tem a curva ROC mais próxima do canto superior esquerdo (alta taxa de verdadeiros positivos e baixa taxa de falsos positivos).

A área sob a curva ROC (AUC, Area Under the Curve) é uma métrica que resume o desempenho do classificador. A AUC varia de 0 a 1, onde 1 representa um classificador perfeito e 0.5 representa um classificador aleatório. Quanto maior a AUC, melhor o classificador.





- **Binarização dos rótulos**: Os rótulos verdadeiros (neste caso os labels 0-9) são binarizados usando a função `label_binarize` do sklearn. Isso é necessário porque a curva ROC e a AUC são calculadas para a classificação binária e precisamos de uma representação binária para a classificação multiclasse.

- **Cálculo da curva ROC e AUC**: O código então entra em um loop sobre o número de classes. Para cada classe, ele calcula a curva ROC e a AUC usando as funções `roc_curve` e `auc` do sklearn, respectivamente.  
 
8. **PR Curve**

A Curva de Precisão-Recall (PR Curve) é uma ferramenta gráfica usada para avaliar o desempenho de um modelo de classificação em termos de precisão e recall. É especialmente útil em situações onde as classes estão muito desbalanceadas.

A Curva de Precisão-Recall é um gráfico de duas dimensões onde a precisão é plotada no eixo Y e o recall (também conhecido como taxa de verdadeiros positivos) é plotado no eixo X.
 
Cada ponto na curva PR representa um par de valores (Recall, Precisão) correspondentes a um limiar de decisão específico. O limiar de decisão é o valor a partir do qual decidimos se uma previsão é classificada como positiva ou negativa. Ao variar o limiar de decisão, obtemos diferentes pares de valores (Recall, Precisão) que formam a curva PR.

A área sob a curva PR (AUC-PR) é uma métrica que resume o desempenho do classificador. Quanto maior a AUC-PR, melhor o classificador. A AUC-PR é especialmente útil quando as classes estão desbalanceadas, pois leva em conta tanto a precisão quanto o recall, ao contrário da AUC-ROC, que pode ser excessivamente otimista em tais situações.
  

In [ ]:


def calculate_roc_auc_pr(model, dataloader):
    model.eval()
    y_test = []
    y_score = []

    # Iterar sobre os dados do dataloader
    for images, labels in dataloader:
        # Fazer a predição do modelo
        outputs = model(images)
        # Aplicar softmax para obter as probabilidades
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        # Adicionar aos arrays
        y_test.append(labels.numpy())
        y_score.append(probabilities.detach().numpy())

    y_test = np.concatenate(y_test)
    y_score = np.concatenate(y_score)

    # Binarizar as labels
    y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    # Calcular ROC para cada classe
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Calcular Precision-Recall para cada classe
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(10):
        precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

    # Plotar ROC e PR para todas as classes em um único gráfico
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for i in range(10):
        axs[0].plot(fpr[i], tpr[i], label='Class %d (area = %0.5f)' % (i, roc_auc[i]))
        axs[1].plot(recall[i], precision[i], label='Class %d (area = %0.5f)' % (i, average_precision[i]))
    
    ####################################################
    axs[0].plot([0, 1], [0, 1], 'k--')
    axs[0].set_xlim([0.0, 1.0])
    axs[0].set_ylim([0.0, 1.05])
    axs[0].set_xlabel('False Positive Rate')
    axs[0].set_ylabel('True Positive Rate')
    axs[0].set_title('Receiver Operating Characteristic')
    axs[0].legend(loc="lower right")

    axs[1].set_xlim([0.0, 1.0])
    axs[1].set_ylim([0.0, 1.05])
    axs[1].set_xlabel('Recall')
    axs[1].set_ylabel('Precision')
    axs[1].set_title('Precision-Recall curve')
    axs[1].legend(loc="lower right")
    
    ####################################################
    #ZOOM
    axs[0].set_xlim([0.0, 0.5])
    axs[0].set_ylim([0.9, 1.05])   
    
    axs[1].set_xlim([0.90, 1.05])
    axs[1].set_ylim([0.5, 1.05])
    ####################################################

    plt.tight_layout()
    plt.show()

    model.train()





In [ ]:

def evaluate_model(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    true_labels = []
    predictions = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.numpy())
            predictions.extend(predicted.numpy())


    # Accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print("Accuracy: ",  round(accuracy, 3))

    # Precision
    precision = precision_score(true_labels, predictions, average='weighted')
    print("Precision: ",  round(precision, 3))

    # Recall
    recall = recall_score(true_labels, predictions, average='weighted')
    print("Recall: ",  round(recall, 3))

    # F1 Score
    f1 = f1_score(true_labels, predictions, average='weighted')
    print("F1 Score: ",  round(f1, 3))
    
#############################################################
#    Está desativado pq não é um problema de classificação binaria (temos 10 classes)
#    # Matthews Correlation Coefficient (MCC)
#    mcc = matthews_corrcoef(true_labels, predictions)
#    print("Matthews Correlation Coefficient: ",  round(mcc, 3))
#############################################################

    # Cohen's Kappa
    kappa = cohen_kappa_score(true_labels, predictions)
    #interpret_kappa
    print("Cohen's Kappa: ",    round(kappa, 3))
    if kappa <= 0:
        print ("Nenhum acordo")
    elif kappa <= 0.20:
        print ("Nenhum a um leve acordo")
    elif kappa <= 0.40:
        print ("Acordo justo")
    elif kappa <= 0.60:
        print ("Acordo moderado")
    elif kappa <= 0.80:
        print ("Acordo substancial")
    elif kappa < 1:
        print ("Acordo quase perfeito")
    elif kappa == 1:
        print ("Acordo perfeito")
    else:
        print ("Valor de Kappa inválido")



    # Classification Report
    report = classification_report(true_labels, predictions)
    print("Classification Report:\n", report)

    # Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)
    #print("Confusion Matrix:\n", cm)
    
    # Plotting Confusion Matrix
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.show()
    
    # Curva de ROC e a AUC-PR
    calculate_roc_auc_pr(model, val_loader)

 




In [ ]:
evaluate_model(model, val_loader)

## Prevendo e identificando erros

In [ ]:

def predict_and_identify_errors(model, val_loader):
    model.eval()  # Defina o modelo para o modo de avaliação
    correct = 0
    total = 0
    wrong_predictions = []

    with torch.no_grad():  # Sem calcular gradientes  
        for images, labels in val_loader:
            outputs = model(images)  # Obtenha as previsões do modelo
            _, predicted = torch.max(outputs.data, 1) 
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Identificar as previsões erradas
            wrong_indices = (predicted != labels).nonzero()[:, 0] 
            #.nonzero() retorna os índices dos elementos diferentes de zero. 
            wrong_images = images[wrong_indices]
            wrong_labels = predicted[wrong_indices]
            true_labels = labels[wrong_indices]
            wrong_predictions.extend(list(zip(wrong_images, wrong_labels, true_labels)))

 
    # Mostra as previsões erradas
    print("Total de previsões erradas:", total - correct )
    print("Previsões erradas:")
    
    # Para 2o imagens, dispostas em 2 linhas e 10 colunas
    n_rows = 3
    n_cols = 10
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols, n_rows+0.3))  # Create a subplot with 1 row and 20 columns
    axs_max=len(list(axs.flat))

    for i, (img, wrong_label, true_label) in enumerate(wrong_predictions[:axs_max]): 
        # Calcula a linha e a coluna atual
        row = i // n_cols
        col = i % n_cols
    
        # Plot das imagens erradas
        axs[row, col].imshow(img.view(28, 28), cmap='binary')
        axs[row, col].axis('off')   
        axs[row, col].set_title(f'P: {wrong_label.item()}, T: {true_label.item()}')  # Defina o título do subplot
    plt.show()  # Show the plot




In [ ]:
predict_and_identify_errors(model, val_loader)


Compreendendo o que está acontecendo em `wrong_predictions.extend(list(zip(wrong_images, wrong_labels, true_labels)))`:

 
1. `zip(wrong_images, wrong_labels, true_labels)`: A função `zip()` pega iteráveis (pode ser zero ou mais), agrega-os em uma tupla e retorna um iterador de tuplas baseado nos iteráveis. Neste caso, `wrong_images`, `wrong_labels` e `true_labels` são agregados juntos em tuplas, onde cada tupla contém uma imagem errada, seu rótulo previsto e seu rótulo verdadeiro.

2. `list(zip(wrong_images, wrong_labels, true_labels))`: A função `list()` é usada para converter o iterador de tuplas retornado por `zip()` em uma lista de tuplas.

3. `wrong_predictions.extend(...)`: A função `extend()` é um método de lista que é usado para adicionar vários elementos ao final da lista existente. Neste caso, está adicionando a lista de tuplas (cada uma contendo uma imagem prevista errada, seu rótulo previsto e seu rótulo verdadeiro) ao final da lista `wrong_predictions`.
 
Compreendendo o calculo da linha e da coluna na grade de subplots:

- `row = i // n_cols`: Esta linha calcula a linha atual na grade de subplots. O operador `//` realiza uma divisão inteira, o que significa que ele divide `i` por `n_cols` e depois arredonda para baixo para o número inteiro mais próximo. Isso é usado para determinar em qual linha do subplot a imagem atual deve ser plotada. Por exemplo, se você tem 10 colunas e está na imagem 11 (índice 10, pois os índices começam em 0), `i // n_cols` seria `10 // 10 = 1`, então a imagem seria plotada na segunda linha.

- `col = i % n_cols`: Esta linha calcula a coluna atual na grade de subplots. O operador `%` calcula o resto da divisão de `i` por `n_cols`. Isso é usado para determinar em qual coluna do subplot a imagem atual deve ser plotada. Por exemplo, se você tem 10 colunas e está na imagem 11 (índice 10), `i % n_cols` seria `10 % 10 = 0`, então a imagem seria plotada na primeira coluna da segunda linha.

Essas duas linhas de código juntas permitem que você percorra uma grade de subplots linha por linha, preenchendo todas as colunas em uma linha antes de passar para a próxima.

# Submissão

In [ ]:

def submission_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad():
        for images in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.numpy())
           # print('oi')
        # Create a DataFrame with the predictions
        df = pd.DataFrame(predictions, columns=['Label'])
        df['Label'] = df['Label'].astype(int)  
        df.index.name = 'ImageId'
        df.index += 1  # Make the index start at 1 instead of 0
    # Save the DataFrame to a CSV file
    df.to_csv('submission.csv')
    return(df)

submission_model(model, test_loader)
 

## Conclusão

Neste notebook, exploramos como treinar e avaliar um modelo de rede neural usando PyTorch. Vimos como calcular várias métricas de desempenho e como visualizá-las e interpretá-las.
 

# FIM